In [1]:
import pandas as pd 
AWM = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/Abusive Towards Women/AWT_train.csv')
len(AWM)

2790

In [2]:
AWM_val = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/Abusive Towards Women/AWT_dev.csv')
len(AWM_val)

598

In [3]:
import numpy as np
label = []
for i in range(0,len(AWM)):
    if AWM['Class'][i] =='Abusive':
        label.append(1)
    else:
        label.append(0)
AWM['label']=np.array(label)

In [4]:
label_val = []
for i in range(0,len(AWM_val)):
    if AWM_val['Class'][i] =='Abusive':
        label_val.append(1)
    else:
        label_val.append(0)
AWM_val['label']=np.array(label_val)

In [5]:
type(AWM_val['Text'][0])

str

In [ ]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['Text'], AWM['label'],AWM_val['Text'], AWM_val['label']
# Load tokenizer
model_name = "xlm-roberta-base" #distlbert-uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

/Users/5036365/Documents/text-recommendation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [8]:
from transformers import AutoModelForSequenceClassification
 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=7,              # Number of epochs
    learning_rate=5e-6,
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/1395 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'loss': 0.7014, 'grad_norm': 2.3903329372406006, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.04}
{'loss': 0.6957, 'grad_norm': 2.638183116912842, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.07}
{'loss': 0.6896, 'grad_norm': 3.6798179149627686, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.11}
{'loss': 0.7053, 'grad_norm': 2.88969087600708, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.14}
{'loss': 0.703, 'grad_norm': 1.2983325719833374, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.18}
{'loss': 0.6876, 'grad_norm': 1.1121445894241333, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.22}
{'loss': 0.6892, 'grad_norm': 2.459695339202881, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.25}
{'loss': 0.6924, 'grad_norm': 1.7212798595428467, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.29}
{'loss': 0.6905, 'grad_norm': 2.4521374702453613, 'learning_rate': 9.000000000000001e-07, 'epoch': 0.32}
{'loss': 0.6907, 'grad_norm': 1.3130757808685303, 'learn

  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.6909783482551575, 'eval_accuracy': 0.507168458781362, 'eval_macro_f1': 0.3428912783751494, 'eval_weighted_f1': 0.34524650676806995, 'eval_runtime': 5.2817, 'eval_samples_per_second': 105.647, 'eval_steps_per_second': 13.253, 'epoch': 1.0}
{'loss': 0.7003, 'grad_norm': 3.077040672302246, 'learning_rate': 2.8000000000000003e-06, 'epoch': 1.0}
{'loss': 0.692, 'grad_norm': 2.6414124965667725, 'learning_rate': 2.9e-06, 'epoch': 1.04}
{'loss': 0.7012, 'grad_norm': 5.285091876983643, 'learning_rate': 3e-06, 'epoch': 1.08}
{'loss': 0.6842, 'grad_norm': 1.8843662738800049, 'learning_rate': 3.1000000000000004e-06, 'epoch': 1.11}
{'loss': 0.6812, 'grad_norm': 7.641088962554932, 'learning_rate': 3.2000000000000003e-06, 'epoch': 1.15}
{'loss': 0.6792, 'grad_norm': 3.2397119998931885, 'learning_rate': 3.3000000000000006e-06, 'epoch': 1.18}
{'loss': 0.7072, 'grad_norm': 5.7334208488464355, 'learning_rate': 3.4000000000000005e-06, 'epoch': 1.22}
{'loss': 0.6731, 'grad_norm': 8.81008243

...

{'eval_loss': 0.6691585779190063, 'eval_accuracy': 0.6684587813620072, 'eval_macro_f1': 0.6581278667351096, 'eval_weighted_f1': 0.658553883833126, 'eval_runtime': 4.9075, 'eval_samples_per_second': 113.704, 'eval_steps_per_second': 14.264, 'epoch': 3.0}
{'loss': 0.542, 'grad_norm': 18.41453742980957, 'learning_rate': 3.1005586592178773e-06, 'epoch': 3.01}
{'loss': 0.5287, 'grad_norm': 4.783198356628418, 'learning_rate': 3.044692737430168e-06, 'epoch': 3.05}
{'loss': 0.5751, 'grad_norm': 19.60125160217285, 'learning_rate': 2.9888268156424584e-06, 'epoch': 3.08}
{'loss': 0.4973, 'grad_norm': 13.729207992553711, 'learning_rate': 2.9329608938547486e-06, 'epoch': 3.12}
{'loss': 0.4631, 'grad_norm': 39.23911666870117, 'learning_rate': 2.877094972067039e-06, 'epoch': 3.15}
{'loss': 0.4293, 'grad_norm': 17.194120407104492, 'learning_rate': 2.8212290502793298e-06, 'epoch': 3.19}
{'loss': 0.5775, 'grad_norm': 27.676698684692383, 'learning_rate': 2.7653631284916204e-06, 'epoch': 3.23}
{'loss': 0.

  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.6389119625091553, 'eval_accuracy': 0.7114695340501792, 'eval_macro_f1': 0.7107924090974939, 'eval_weighted_f1': 0.7108927239052991, 'eval_runtime': 4.9184, 'eval_samples_per_second': 113.452, 'eval_steps_per_second': 14.232, 'epoch': 4.0}
{'loss': 0.5484, 'grad_norm': 47.41437530517578, 'learning_rate': 1.5363128491620113e-06, 'epoch': 4.01}
{'loss': 0.4893, 'grad_norm': 8.93497085571289, 'learning_rate': 1.4804469273743019e-06, 'epoch': 4.05}
{'loss': 0.5275, 'grad_norm': 45.3161735534668, 'learning_rate': 1.4245810055865922e-06, 'epoch': 4.09}
{'loss': 0.4761, 'grad_norm': 31.78418731689453, 'learning_rate': 1.3687150837988828e-06, 'epoch': 4.12}
{'loss': 0.5174, 'grad_norm': 53.7054557800293, 'learning_rate': 1.3128491620111732e-06, 'epoch': 4.16}
{'loss': 0.3359, 'grad_norm': 28.486665725708008, 'learning_rate': 1.2569832402234638e-06, 'epoch': 4.19}
{'loss': 0.5365, 'grad_norm': 49.37507629394531, 'learning_rate': 1.2011173184357544e-06, 'epoch': 4.23}
{'loss': 0.2

  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.6474334001541138, 'eval_accuracy': 0.7275985663082437, 'eval_macro_f1': 0.7274725274725274, 'eval_weighted_f1': 0.7275145404177662, 'eval_runtime': 4.6329, 'eval_samples_per_second': 120.444, 'eval_steps_per_second': 15.109, 'epoch': 5.0}
{'train_runtime': 580.5651, 'train_samples_per_second': 19.223, 'train_steps_per_second': 2.403, 'train_loss': 0.5677633374395337, 'epoch': 5.0}


In [9]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=7,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2443 [00:00<?, ?it/s]

{'loss': 0.7125, 'grad_norm': 1.3030606508255005, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.7206, 'grad_norm': 6.668478965759277, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}
{'loss': 0.7123, 'grad_norm': 5.97066593170166, 'learning_rate': 3e-06, 'epoch': 0.09}
{'loss': 0.7201, 'grad_norm': 3.996624231338501, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.11}
{'loss': 0.694, 'grad_norm': 1.6091792583465576, 'learning_rate': 5e-06, 'epoch': 0.14}
{'loss': 0.6923, 'grad_norm': 3.787095308303833, 'learning_rate': 6e-06, 'epoch': 0.17}
{'loss': 0.7364, 'grad_norm': 2.7237818241119385, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.6905, 'grad_norm': 3.586463212966919, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.23}
{'loss': 0.7154, 'grad_norm': 3.5877699851989746, 'learning_rate': 9e-06, 'epoch': 0.26}
{'loss': 0.7155, 'grad_norm': 8.306807518005371, 'learning_rate': 1e-05, 'epoch': 0.29}
{'loss': 0.6733, 'grad_norm': 2.87480

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.6855587363243103, 'eval_accuracy': 0.5869565217391305, 'eval_macro_f1': 0.50364104993968, 'eval_weighted_f1': 0.5179237022481571, 'eval_runtime': 5.1152, 'eval_samples_per_second': 116.905, 'eval_steps_per_second': 14.662, 'epoch': 1.0}
{'loss': 0.7601, 'grad_norm': 4.96051549911499, 'learning_rate': 3.5e-05, 'epoch': 1.0}
{'loss': 0.6926, 'grad_norm': 5.489762306213379, 'learning_rate': 3.6e-05, 'epoch': 1.03}
{'loss': 0.7079, 'grad_norm': 3.0916621685028076, 'learning_rate': 3.7e-05, 'epoch': 1.06}
{'loss': 0.6184, 'grad_norm': 23.012964248657227, 'learning_rate': 3.8e-05, 'epoch': 1.09}
{'loss': 0.6185, 'grad_norm': 130.84742736816406, 'learning_rate': 3.9000000000000006e-05, 'epoch': 1.12}
{'loss': 0.7675, 'grad_norm': 3.9598336219787598, 'learning_rate': 4e-05, 'epoch': 1.15}
{'loss': 0.7256, 'grad_norm': 6.200557231903076, 'learning_rate': 4.1e-05, 'epoch': 1.17}
{'loss': 0.715, 'grad_norm': 3.0235562324523926, 'learning_rate': 4.2e-05, 'epoch': 1.2}
{'loss': 0.74

...

{'eval_loss': 0.5283880829811096, 'eval_accuracy': 0.7558528428093646, 'eval_macro_f1': 0.7556314375279893, 'eval_weighted_f1': 0.7551148252047802, 'eval_runtime': 5.1548, 'eval_samples_per_second': 116.008, 'eval_steps_per_second': 14.55, 'epoch': 5.0}
{'loss': 0.4618, 'grad_norm': 14.869614601135254, 'learning_rate': 1.7833247555326814e-05, 'epoch': 5.01}
{'loss': 0.4118, 'grad_norm': 7.133627414703369, 'learning_rate': 1.7575913535769428e-05, 'epoch': 5.04}
{'loss': 0.4594, 'grad_norm': 3.9304745197296143, 'learning_rate': 1.7318579516212042e-05, 'epoch': 5.07}
{'loss': 0.3852, 'grad_norm': 4.245253562927246, 'learning_rate': 1.706124549665466e-05, 'epoch': 5.1}
{'loss': 0.4606, 'grad_norm': 1.6010116338729858, 'learning_rate': 1.6803911477097274e-05, 'epoch': 5.13}
{'loss': 0.3697, 'grad_norm': 1.6295545101165771, 'learning_rate': 1.654657745753989e-05, 'epoch': 5.16}
{'loss': 0.3384, 'grad_norm': 4.387369155883789, 'learning_rate': 1.6289243437982503e-05, 'epoch': 5.19}
{'loss': 0

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.6099260449409485, 'eval_accuracy': 0.7525083612040134, 'eval_macro_f1': 0.7523726385531382, 'eval_weighted_f1': 0.7519654706005124, 'eval_runtime': 5.05, 'eval_samples_per_second': 118.415, 'eval_steps_per_second': 14.851, 'epoch': 6.0}
{'loss': 0.281, 'grad_norm': 12.648601531982422, 'learning_rate': 8.826556870818323e-06, 'epoch': 6.02}
{'loss': 0.3675, 'grad_norm': 6.473410129547119, 'learning_rate': 8.569222851260937e-06, 'epoch': 6.05}
{'loss': 0.1967, 'grad_norm': 60.51615905761719, 'learning_rate': 8.311888831703551e-06, 'epoch': 6.07}
{'loss': 0.2839, 'grad_norm': 0.8277703523635864, 'learning_rate': 8.054554812146166e-06, 'epoch': 6.1}
{'loss': 0.4767, 'grad_norm': 0.8592621684074402, 'learning_rate': 7.79722079258878e-06, 'epoch': 6.13}
{'loss': 0.5279, 'grad_norm': 32.45942306518555, 'learning_rate': 7.539886773031394e-06, 'epoch': 6.16}
{'loss': 0.3238, 'grad_norm': 8.864876747131348, 'learning_rate': 7.282552753474009e-06, 'epoch': 6.19}
{'loss': 0.2511, 'g

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.7229607105255127, 'eval_accuracy': 0.7692307692307693, 'eval_macro_f1': 0.7692075353499195, 'eval_weighted_f1': 0.7693701725158679, 'eval_runtime': 5.2156, 'eval_samples_per_second': 114.656, 'eval_steps_per_second': 14.38, 'epoch': 7.0}
{'train_runtime': 992.2913, 'train_samples_per_second': 19.682, 'train_steps_per_second': 2.462, 'train_loss': 0.5255951188501646, 'epoch': 7.0}


In [34]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=7,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/2443 [00:00<?, ?it/s]

{'loss': 0.7055, 'grad_norm': 2.4591064453125, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.698, 'grad_norm': 5.322478294372559, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}
{'loss': 0.7023, 'grad_norm': 4.648119926452637, 'learning_rate': 3e-06, 'epoch': 0.09}
{'loss': 0.6991, 'grad_norm': 2.17541241645813, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.11}
{'loss': 0.7008, 'grad_norm': 2.655975103378296, 'learning_rate': 5e-06, 'epoch': 0.14}
{'loss': 0.6885, 'grad_norm': 4.5047125816345215, 'learning_rate': 6e-06, 'epoch': 0.17}
{'loss': 0.7106, 'grad_norm': 1.4660624265670776, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.6978, 'grad_norm': 2.2469043731689453, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.23}
{'loss': 0.688, 'grad_norm': 2.2223477363586426, 'learning_rate': 9e-06, 'epoch': 0.26}
{'loss': 0.688, 'grad_norm': 6.558040142059326, 'learning_rate': 1e-05, 'epoch': 0.29}
{'loss': 0.697, 'grad_norm': 2.8226201534

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.7060222029685974, 'eval_accuracy': 0.46488294314381273, 'eval_macro_f1': 0.317351598173516, 'eval_weighted_f1': 0.2950626899405934, 'eval_runtime': 5.1504, 'eval_samples_per_second': 116.106, 'eval_steps_per_second': 14.562, 'epoch': 1.0}
{'loss': 0.7443, 'grad_norm': 5.256906986236572, 'learning_rate': 3.5e-05, 'epoch': 1.0}
{'loss': 0.6951, 'grad_norm': 1.6575927734375, 'learning_rate': 3.6e-05, 'epoch': 1.03}
{'loss': 0.6931, 'grad_norm': 1.7412819862365723, 'learning_rate': 3.7e-05, 'epoch': 1.06}
{'loss': 0.703, 'grad_norm': 3.2224740982055664, 'learning_rate': 3.8e-05, 'epoch': 1.09}
{'loss': 0.7133, 'grad_norm': 4.08530855178833, 'learning_rate': 3.9000000000000006e-05, 'epoch': 1.12}
{'loss': 0.7048, 'grad_norm': 1.4301460981369019, 'learning_rate': 4e-05, 'epoch': 1.15}
{'loss': 0.7033, 'grad_norm': 3.069645643234253, 'learning_rate': 4.1e-05, 'epoch': 1.17}
{'loss': 0.7094, 'grad_norm': 1.5487245321273804, 'learning_rate': 4.2e-05, 'epoch': 1.2}
{'loss': 0.692

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.6990594267845154, 'eval_accuracy': 0.46488294314381273, 'eval_macro_f1': 0.317351598173516, 'eval_weighted_f1': 0.2950626899405934, 'eval_runtime': 5.0806, 'eval_samples_per_second': 117.703, 'eval_steps_per_second': 14.762, 'epoch': 2.0}
{'loss': 0.6967, 'grad_norm': 5.196981906890869, 'learning_rate': 4.485331960885229e-05, 'epoch': 2.01}
{'loss': 0.6967, 'grad_norm': 2.2718303203582764, 'learning_rate': 4.4595985589294906e-05, 'epoch': 2.03}
{'loss': 0.6997, 'grad_norm': 3.4045960903167725, 'learning_rate': 4.4338651569737524e-05, 'epoch': 2.06}
{'loss': 0.7111, 'grad_norm': 3.719268798828125, 'learning_rate': 4.4081317550180135e-05, 'epoch': 2.09}


KeyboardInterrupt: 